### Star Ratings Results
- Input from AimBig notebook, stat_regression 
- 6 steps of the star rating results
- GroupBy object = (Contract ID, ESA Code, Speciality_site_type_code)

In [1]:
import datetime as dt
import re
import json
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from datetime import date, timedelta
from datetime import datetime
import warnings
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sn
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# reading in file with numerator/denominator for each participant, performance measure against contract
df_merged = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_merge_final.csv')
# creating groupBy object 
df_merged = df_merged.drop_duplicates()
#df_group = df_merged.groupby(['CONTRACT_ID', 'ESA_CODE', 'SPECIALIST_SITE_TYPE_CODE'])

df_merged.columns

df_merged = df_merged.drop(columns=['Unnamed: 0'])

Columns missing due to no outcomes = 'Num_26_path', 'Num_26_wrkast', 'Den_26_path', 'Den_26_wrkast', 'Den_52_path', 'Num_52_path'
- set as 0 

In [2]:
df_merged

,JOB_SEEKER_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,Den_26_wrkast,Num_52_full,Den_52_full,Num_52_path,Den_52_path
0,9386301003,0,0,AimBig Employment WOY WOY,0212812K,AALL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8653691003,0,0,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
2,7870290004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
3,6638331004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2580218019,0,0,AimBig Employment LEICHHARDT,0212900K,AALL,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,1922050119,0,0,AimBig Employment LEICHHARDT,0212900K,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
1449,7877870004,0,0,AimBig Employment DALBY,0212897K,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
1450,3825170009,0,4,AimBig Employment FRANKSTON,0212818F,MENH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1451,4123062109,0,0,AimBig Employment ROCKHAMPTON CITY,0212801H,MUSK,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0


In [3]:
print(df_merged.isna().sum())

JOB_SEEKER_ID                0
Num_13                       0
Den_13                       0
Site_Name                    0
Contract_ID                  0
specialist_site_type_code    0
Num_26_full                  0
Den_26_full                  0
Num_26_path                  0
Den_26_path                  0
Num_26_wrkast                0
Den_26_wrkast                0
Num_52_full                  0
Den_52_full                  0
Num_52_path                  0
Den_52_path                  0
dtype: int64


### Step 1) Calculating Actual Performance
#### Actual Performance
groupBy key = (Contract ID, ESA Code, Speciality_site_type_code)

##### Pre-Quarter

- Count NaNs -> place as 0

In [4]:
df_merged = df_merged.fillna(value = 0)
df_merged

,JOB_SEEKER_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,Den_26_wrkast,Num_52_full,Den_52_full,Num_52_path,Den_52_path
0,9386301003,0,0,AimBig Employment WOY WOY,0212812K,AALL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8653691003,0,0,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
2,7870290004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
3,6638331004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2580218019,0,0,AimBig Employment LEICHHARDT,0212900K,AALL,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,1922050119,0,0,AimBig Employment LEICHHARDT,0212900K,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
1449,7877870004,0,0,AimBig Employment DALBY,0212897K,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
1450,3825170009,0,4,AimBig Employment FRANKSTON,0212818F,MENH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1451,4123062109,0,0,AimBig Employment ROCKHAMPTON CITY,0212801H,MUSK,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0


In [5]:
df_merged.columns

Index(['JOB_SEEKER_ID', 'Num_13', 'Den_13', 'Site_Name', 'Contract_ID',
       'specialist_site_type_code', 'Num_26_full', 'Den_26_full',
       'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast',
       'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'],
      dtype='object')

In [6]:
df_merged = df_merged.rename(columns={'Num_13' : 'Num_13_full', 'Den_13' : 'Den_13_full'})

### Aggregating to Site Level 

In [7]:
df_group = df_merged.groupby(['Contract_ID', 'Site_Name',  'specialist_site_type_code'])['Num_13_full', 'Den_13_full', 'Site_Name', 'Contract_ID',
       'specialist_site_type_code', 'Num_26_full', 'Den_26_full',
       'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast',
       'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()


df_group.to_csv('df_group_fin.csv')
df_group['actual_13_full'] = df_group['Num_13_full']/df_group['Den_13_full'] * 100
df_group['actual_26_full'] = df_group['Num_26_full']/df_group['Den_26_full'] * 100

df_group['actual_26_path'] = df_group['Num_26_path']/df_group['Den_26_path'] * 100
df_group['actual_26_wrkast'] = df_group['Num_26_wrkast']/df_group['Den_26_wrkast'] * 100
df_group['actual_52_full'] = df_group['Num_52_full']/df_group['Den_52_full'] * 100
df_group['actual_52_path'] = df_group['Num_52_path']/df_group['Den_52_path'] * 100




#### Renaming df_group -> df_pre_actual

In [8]:
df_pre_actual = df_group
df_pre_actual.to_csv('C:/Users/AManalo/star_ratings_new/Output Files/test_sites.csv')

In [9]:
def get_actPerf(type, df_group):
    '''Input:
        - type: 'pre' or 'post' quarter
        - df_group: groupBy object that contains site participants with num/den
        Output:
        - returns dataframe with pre/post quarter actual performance for each Contract'''
    
    if type == 'pre':
        df_group['preAct13_full'] = 0 # initialise pre_actual_rate column
        df_group['preAct26_path'] = 0
        df_group['preAct26_full'] = 0
        df_group['preAct26_wrkast'] = 0
        df_group['preAct52_full'] = 0
        df_group['preAct52_path'] = 0
        

    elif type == 'post':
        df_group['postAct13_full'] = 0 # initialise post_actual_rate column
        df_group['postAct26_path'] = 0
        df_group['postAct26_full'] = 0
        df_group['postAct26_wrkast'] = 0
        df_group['postAct52_full'] = 0
        df_group['postAct52_path'] = 0

    

    return df_group

##### Post-Quarter

In [10]:
# reading in the merged file
df_merge_post = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_merge_post.csv')
df_merge_post = df_merge_post.fillna(value = 0)
df_merge_post = df_merge_post.rename(columns={'CONTRACT_ID' : 'Contract_ID', 'SITE_CODE' : 'Site_Code', 'SITE_DESCRIPTION': 'Site_Name', 'SPECIALIST_SITE_TYPE_CODE' : 'specialist_site_type_code'})
df_merge_post.head(5)

,Unnamed: 0,Program,JOB_SEEKER_ID,Contract_ID,Site_Code,Site_Name,ESA Code,ESA Name,State,specialist_site_type_code,...,Num_26_path_exp,Num_26_wrkast,Den_26_wrkast,Num_26_wrkast_exp,Num_52_full,Den_52_full,Num_52_full_exp,Num_52_path,Den_52_path,Num_52_path_exp
0,0,ESS,168470,0212898K,AB31,AimBig Employment AUBURN,4CWS,Northern Sydney,NSW,AALL,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1,DMS,451780,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,QLD,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
2,2,DMS,3606070,0212846K,MM07,AimBig Employment BURWOOD,4INW,South East Sydney,NSW,MENH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,DMS,618010,0212823C,MN07,AimBig Employment SUNSHINE,4WES,Western Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,DMS,630040,0212831C,MM67,AimBig Employment ROCKINGHAM,4CWM,Western Australia,WA,MENH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


This is WITHOUT 52 week outcomes -> ONLY FOR TESTING 13 and 26 week

In [11]:
df_merge_post = df_merge_post.rename(columns={'Num_13' : 'Num_13_full', 'Den_13' : 'Den_13_full'})

In [12]:
df_merge_post

,Unnamed: 0,Program,JOB_SEEKER_ID,Contract_ID,Site_Code,Site_Name,ESA Code,ESA Name,State,specialist_site_type_code,...,Num_26_path_exp,Num_26_wrkast,Den_26_wrkast,Num_26_wrkast_exp,Num_52_full,Den_52_full,Num_52_full_exp,Num_52_path,Den_52_path,Num_52_path_exp
0,0,ESS,168470,0212898K,AB31,AimBig Employment AUBURN,4CWS,Northern Sydney,NSW,AALL,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1,DMS,451780,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,QLD,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
2,2,DMS,3606070,0212846K,MM07,AimBig Employment BURWOOD,4INW,South East Sydney,NSW,MENH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,DMS,618010,0212823C,MN07,AimBig Employment SUNSHINE,4WES,Western Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,DMS,630040,0212831C,MM67,AimBig Employment ROCKINGHAM,4CWM,Western Australia,WA,MENH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5307,5308,DMS,2730016019,0212841E,Y947,AimBig Employment MERRYLANDS,4CWS,Northern Sydney,NSW,AALL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5308,5309,DMS,2696040,0212857B,ML77,AimBig Employment WOLLONGONG,4WOL,South East Sydney,NSW,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5309,5310,DMS,5892574007,0212825E,MM77,AimBig Employment MELBOURNE,4YAR,Northern Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5310,5311,DMS,832617003,0212823C,YA97,AimBig Employment Pty Ltd ST ALBANS,4WES,Western Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_group = df_merge_post.groupby(['Contract_ID', 'Site_Name', 'specialist_site_type_code', 'JOB_SEEKER_ID'])['Num_13_full', 'Den_13_full', 'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast', 'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()
df_group = df_merge_post.groupby(['Contract_ID', 'Site_Name', 'specialist_site_type_code'])['Num_13_full', 'Den_13_full', 'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast', 'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()


df_group.to_csv('C:/Users/AManalo/star_ratings_new/df_group_fin.csv')

Calculating Actual Performance

In [14]:
# renaming df_13 ->



df_group['actual_13_full'] = df_group['Num_13_full']/df_group['Den_13_full'] * 100
df_group['actual_26_full'] = df_group['Num_26_full']/df_group['Den_26_full'] * 100

df_group['actual_26_path'] = df_group['Num_26_path']/df_group['Den_26_path'] * 100
df_group['actual_26_wrkast'] = df_group['Num_26_wrkast']/df_group['Den_26_wrkast'] * 100
df_group['actual_52_full'] = df_group['Num_52_full']/df_group['Den_52_full'] * 100
df_group['actual_52_path'] = df_group['Num_52_path']/df_group['Den_52_path'] * 100

df_group = df_group.fillna(0) # if numerator is a 0, then division by a 0 results in a NaN -> but by documentation, should be 0.

df_group.to_csv('C:/Users/AManalo/star_ratings_new/df_group_test.csv')

In [15]:
df_group

Num_13_full  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               2.0   
            AimBig Employment PROSPECT   MENH                              12.0   
0212790K    AimBig Employment NORWOOD    MUSK                               1.0   
            AimBig Employment PROSPECT   MUSK                               3.0   
0212791K    AimBig Employment MILE END   MENH                               8.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               1.0   
0212904D    AimBig Employment HURSTVILLE AALL                              13.0   
            AimBig Employment MIRANDA    AALL                               6.0   
            AimBig Employment ROCKDALE   AALL                               4.0   

                                                                    Den_13_full  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               2.0   
            AimBig Employment PROSPECT   MENH                              18.0   
0212790K    AimBig Employment NORWOOD    MUSK                               1.0   
            AimBig Employment PROSPECT   MUSK                               4.0   
0212791K    AimBig Employment MILE END   MENH                              13.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               1.0   
0212904D    AimBig Employment HURSTVILLE AALL                              19.0   
            AimBig Employment MIRANDA    AALL                               9.0   
            AimBig Employment ROCKDALE   AALL                               7.0   

                                                                    Num_26_full  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               1.0   
            AimBig Employment PROSPECT   MENH                               8.0   
0212790K    AimBig Employment NORWOOD    MUSK                               1.0   
            AimBig Employment PROSPECT   MUSK                               1.0   
0212791K    AimBig Employment MILE END   MENH                               6.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               0.0   
0212904D    AimBig Employment HURSTVILLE AALL                               7.0   
            AimBig Employment MIRANDA    AALL                               3.0   
            AimBig Employment ROCKDALE   AALL                               0.0   

                                                                    Den_26_full  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               1.0   
            AimBig Employment PROSPECT   MENH                              13.0   
0212790K    AimBig Employment NORWOOD    MUSK                               1.0   
            AimBig Employment PROSPECT   MUSK                               1.0   
0212791K    AimBig Employment MILE END   MENH                               9.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               0.0  

### Validating Contracts

## Step 2: Expected Performance

Using statistical regression, which takes account of variable Participant and labour market characteristics and the number of Participants in the denominator post-quarterisation, the Star Ratings model calculates the number of outcomes that the Contract could be expected to achieve.

In [16]:

df_group.columns

Index(['Num_13_full', 'Den_13_full', 'Num_26_full', 'Den_26_full',
       'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast',
       'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path',
       'actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path'],
      dtype='object')

In [17]:
df_merge_post

,Unnamed: 0,Program,JOB_SEEKER_ID,Contract_ID,Site_Code,Site_Name,ESA Code,ESA Name,State,specialist_site_type_code,...,Num_26_path_exp,Num_26_wrkast,Den_26_wrkast,Num_26_wrkast_exp,Num_52_full,Den_52_full,Num_52_full_exp,Num_52_path,Den_52_path,Num_52_path_exp
0,0,ESS,168470,0212898K,AB31,AimBig Employment AUBURN,4CWS,Northern Sydney,NSW,AALL,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1,DMS,451780,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,QLD,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
2,2,DMS,3606070,0212846K,MM07,AimBig Employment BURWOOD,4INW,South East Sydney,NSW,MENH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,DMS,618010,0212823C,MN07,AimBig Employment SUNSHINE,4WES,Western Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,DMS,630040,0212831C,MM67,AimBig Employment ROCKINGHAM,4CWM,Western Australia,WA,MENH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5307,5308,DMS,2730016019,0212841E,Y947,AimBig Employment MERRYLANDS,4CWS,Northern Sydney,NSW,AALL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5308,5309,DMS,2696040,0212857B,ML77,AimBig Employment WOLLONGONG,4WOL,South East Sydney,NSW,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5309,5310,DMS,5892574007,0212825E,MM77,AimBig Employment MELBOURNE,4YAR,Northern Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5310,5311,DMS,832617003,0212823C,YA97,AimBig Employment Pty Ltd ST ALBANS,4WES,Western Victoria,VIC,MUSK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Get expected columns from df_merged
df_merge_post.reset_index()
df_merge_post.columns

df_exp = df_merge_post[['JOB_SEEKER_ID', 'Contract_ID',  'Site_Name', 'specialist_site_type_code', 'Num_13_full_exp', 'Num_26_full_exp', 'Num_26_path_exp', 'Num_26_wrkast_exp', 'Num_52_full_exp', 'Num_52_path_exp']]
       

Merging df_exp with the denominator from post quarterisation outcomes

In [19]:
df_merge_post.columns

Index(['Unnamed: 0', 'Program', 'JOB_SEEKER_ID', 'Contract_ID', 'Site_Code',
       'Site_Name', 'ESA Code', 'ESA Name', 'State',
       'specialist_site_type_code', 'Num_13_full', 'Den_13_full',
       'Num_13_full_exp', 'index', 'Num_26_full', 'Den_26_full',
       'Num_26_full_exp', 'Num_26_path', 'Den_26_path', 'Num_26_path_exp',
       'Num_26_wrkast', 'Den_26_wrkast', 'Num_26_wrkast_exp', 'Num_52_full',
       'Den_52_full', 'Num_52_full_exp', 'Num_52_path', 'Den_52_path',
       'Num_52_path_exp'],
      dtype='object')

In [20]:
df_exp

,JOB_SEEKER_ID,Contract_ID,Site_Name,specialist_site_type_code,Num_13_full_exp,Num_26_full_exp,Num_26_path_exp,Num_26_wrkast_exp,Num_52_full_exp,Num_52_path_exp
0,168470,0212898K,AimBig Employment AUBURN,AALL,0.0,0.0,0.0,0.0,0.0,0.0
1,451780,0212875D,AimBig Employment CALOUNDRA,MUSK,1.0,1.0,0.0,0.0,1.0,1.0
2,3606070,0212846K,AimBig Employment BURWOOD,MENH,0.0,0.0,0.0,0.0,0.0,0.0
3,618010,0212823C,AimBig Employment SUNSHINE,MUSK,0.0,0.0,0.0,0.0,0.0,0.0
4,630040,0212831C,AimBig Employment ROCKINGHAM,MENH,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
5307,2730016019,0212841E,AimBig Employment MERRYLANDS,AALL,0.0,0.0,0.0,0.0,0.0,0.0
5308,2696040,0212857B,AimBig Employment WOLLONGONG,MUSK,0.0,0.0,0.0,0.0,0.0,0.0
5309,5892574007,0212825E,AimBig Employment MELBOURNE,MUSK,0.0,0.0,0.0,0.0,0.0,0.0
5310,832617003,0212823C,AimBig Employment Pty Ltd ST ALBANS,MUSK,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df_merge_post
df_post_den = df_merge_post[["JOB_SEEKER_ID", "Den_13_full", "Den_26_full", "Den_26_path", "Den_26_wrkast", "Den_52_full", "Den_52_path"]]
df_exp_per = df_exp.merge(df_post_den, on=['JOB_SEEKER_ID'], how='inner')


df_exp_per = df_exp_per.groupby(['Contract_ID', 'Site_Name',  'specialist_site_type_code'])[ "Den_13_full", "Den_26_full", "Den_26_path", "Den_26_wrkast", "Den_52_full", "Den_52_path", 'Num_13_full_exp', 'Num_26_full_exp', 'Num_52_full_exp', 'Num_52_path_exp', 'Num_26_wrkast_exp', 'Num_26_path_exp'].sum()



Calculating expected performance rate
- for each performance measure

In [23]:

df_exp_per['exPer_13_full'] = df_exp_per['Den_13_full']/df_exp_per['Num_13_full_exp']
df_exp_per['exPer_26_full'] = df_exp_per['Den_26_full']/df_exp_per['Num_26_full_exp']
df_exp_per['exPer_26_path'] = df_exp_per['Den_26_path']/df_exp_per['Num_26_path_exp']
df_exp_per['exPer_26_wrkast'] = df_exp_per['Den_26_wrkast']/df_exp_per['Num_26_wrkast_exp']
df_exp_per['exPer_52_full'] = df_exp_per['Den_52_full']/df_exp_per['Num_52_full_exp']
df_exp_per['exPer_52_path'] = df_exp_per['Den_52_path']/df_exp_per['Num_52_path_exp']


df_exp_per

Den_13_full  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               2.0   
            AimBig Employment PROSPECT   MENH                              18.0   
0212790K    AimBig Employment NORWOOD    MUSK                               1.0   
            AimBig Employment PROSPECT   MUSK                               4.0   
0212791K    AimBig Employment MILE END   MENH                              13.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               1.0   
0212904D    AimBig Employment HURSTVILLE AALL                              19.0   
            AimBig Employment MIRANDA    AALL                               9.0   
            AimBig Employment ROCKDALE   AALL                               7.0   

                                                                    Den_26_full  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               1.0   
            AimBig Employment PROSPECT   MENH                              13.0   
0212790K    AimBig Employment NORWOOD    MUSK                               1.0   
            AimBig Employment PROSPECT   MUSK                               1.0   
0212791K    AimBig Employment MILE END   MENH                               9.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               0.0   
0212904D    AimBig Employment HURSTVILLE AALL                              10.0   
            AimBig Employment MIRANDA    AALL                               5.0   
            AimBig Employment ROCKDALE   AALL                               0.0   

                                                                    Den_26_path  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               0.0   
            AimBig Employment PROSPECT   MENH                               5.0   
0212790K    AimBig Employment NORWOOD    MUSK                               0.0   
            AimBig Employment PROSPECT   MUSK                               0.0   
0212791K    AimBig Employment MILE END   MENH                               4.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               0.0   
0212904D    AimBig Employment HURSTVILLE AALL                               3.0   
            AimBig Employment MIRANDA    AALL                               2.0   
            AimBig Employment ROCKDALE   AALL                               0.0   

                                                                    Den_26_wrkast  \
Contract_ID Site_Name                    specialist_site_type_code                  
0212789F    AimBig Employment NORWOOD    MENH                                 0.0   
            AimBig Employment PROSPECT   MENH                                 0.0   
0212790K    AimBig Employment NORWOOD    MUSK                                 0.0   
            AimBig Employment PROSPECT   MUSK                                 0.0   
0212791K    AimBig Employment MILE END   MENH                                 0.0   
...                                                                           ...   
0212903C    Aimbig Employment RICHMOND   AALL                                 0.0   
            Aimbig Employment WINDSOR    AALL                  

## Step 3: Calculate the Performance Measure Scores (Ratio of Actual Outcomes : Expected Outcomes)

Combined 13, 25, 52 week scores are calculated based on the weightings of each sub type measure. 
Actual rate: actual rate (post-quarter)
Expected rate: expected performance rate


To-Fix: Need to adjust the weights for each performance measure IF the denominator in the pre-quarter outcomes are 0 or very small. 

- ESS: min 20 participants (combined) in the pre-quarterisation denominators for the following: 13-week full outcomes (min 5 participants), ongoing support  
- DMS: min 20 participants in the pre-quarterisation denominator for 13 week full outcomes measure

In [24]:
df_group.reset_index()

,Contract_ID,Site_Name,specialist_site_type_code,Num_13_full,Den_13_full,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,...,Num_52_full,Den_52_full,Num_52_path,Den_52_path,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path
0,0212789F,AimBig Employment NORWOOD,MENH,2.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.000000,100.000000,0.0,0.0,0.000000,0.0
1,0212789F,AimBig Employment PROSPECT,MENH,12.0,18.0,8.0,13.0,0.0,5.0,0.0,...,3.0,9.0,0.0,0.0,66.666667,61.538462,0.0,0.0,33.333333,0.0
2,0212790K,AimBig Employment NORWOOD,MUSK,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.000000,100.000000,0.0,0.0,0.000000,0.0
3,0212790K,AimBig Employment PROSPECT,MUSK,3.0,4.0,1.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,75.000000,100.000000,0.0,0.0,100.000000,0.0
4,0212791K,AimBig Employment MILE END,MENH,8.0,13.0,6.0,9.0,1.0,4.0,0.0,...,1.0,4.0,0.0,0.0,61.538462,66.666667,25.0,0.0,25.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,0212903C,Aimbig Employment RICHMOND,AALL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
209,0212903C,Aimbig Employment WINDSOR,AALL,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.000000,0.000000,0.0,0.0,0.000000,0.0
210,0212904D,AimBig Employment HURSTVILLE,AALL,13.0,19.0,7.0,10.0,0.0,3.0,0.0,...,0.0,2.0,0.0,0.0,68.421053,70.000000,0.0,0.0,0.000000,0.0
211,0212904D,AimBig Employment MIRANDA,AALL,6.0,9.0,3.0,5.0,0.0,2.0,0.0,...,0.0,2.0,0.0,0.0,66.666667,60.000000,0.0,0.0,0.000000,0.0


In [25]:
df_exp_per.to_csv('C:/Users/AManalo/star_ratings_new/expected_outcomes.csv')

In [26]:
df_exp_per

Den_13_full  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               2.0   
            AimBig Employment PROSPECT   MENH                              18.0   
0212790K    AimBig Employment NORWOOD    MUSK                               1.0   
            AimBig Employment PROSPECT   MUSK                               4.0   
0212791K    AimBig Employment MILE END   MENH                              13.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               1.0   
0212904D    AimBig Employment HURSTVILLE AALL                              19.0   
            AimBig Employment MIRANDA    AALL                               9.0   
            AimBig Employment ROCKDALE   AALL                               7.0   

                                                                    Den_26_full  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               1.0   
            AimBig Employment PROSPECT   MENH                              13.0   
0212790K    AimBig Employment NORWOOD    MUSK                               1.0   
            AimBig Employment PROSPECT   MUSK                               1.0   
0212791K    AimBig Employment MILE END   MENH                               9.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               0.0   
0212904D    AimBig Employment HURSTVILLE AALL                              10.0   
            AimBig Employment MIRANDA    AALL                               5.0   
            AimBig Employment ROCKDALE   AALL                               0.0   

                                                                    Den_26_path  \
Contract_ID Site_Name                    specialist_site_type_code                
0212789F    AimBig Employment NORWOOD    MENH                               0.0   
            AimBig Employment PROSPECT   MENH                               5.0   
0212790K    AimBig Employment NORWOOD    MUSK                               0.0   
            AimBig Employment PROSPECT   MUSK                               0.0   
0212791K    AimBig Employment MILE END   MENH                               4.0   
...                                                                         ...   
0212903C    Aimbig Employment RICHMOND   AALL                               0.0   
            Aimbig Employment WINDSOR    AALL                               0.0   
0212904D    AimBig Employment HURSTVILLE AALL                               3.0   
            AimBig Employment MIRANDA    AALL                               2.0   
            AimBig Employment ROCKDALE   AALL                               0.0   

                                                                    Den_26_wrkast  \
Contract_ID Site_Name                    specialist_site_type_code                  
0212789F    AimBig Employment NORWOOD    MENH                                 0.0   
            AimBig Employment PROSPECT   MENH                                 0.0   
0212790K    AimBig Employment NORWOOD    MUSK                                 0.0   
            AimBig Employment PROSPECT   MUSK                                 0.0   
0212791K    AimBig Employment MILE END   MENH                                 0.0   
...                                                                           ...   
0212903C    Aimbig Employment RICHMOND   AALL                                 0.0   
            Aimbig Employment WINDSOR    AALL                  

In [27]:
# subsetting the post-quarter actual rates for each contract
df_actPer = df_group.reset_index() # converting groupBy obj -> dataframe for subsetting
df_actPer.columns
df_actPer = df_actPer[['Contract_ID',  'Site_Name',  'specialist_site_type_code', 'actual_13_full',
       'actual_26_full', 'actual_26_path', 'actual_26_wrkast',
       'actual_52_full', 'actual_52_path']]

# subsetting the expected performance for each contract
df_exp_per = df_exp_per.reset_index()
df_exp_per.columns
df_exp_per = df_exp_per[['Contract_ID','Site_Name', 'specialist_site_type_code', 'exPer_13_full', 'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast',
       'exPer_52_full', 'exPer_52_path']]

# joining the two dataframes
df_perMea = df_actPer.merge(df_exp_per, on=['Contract_ID',  'Site_Name',  'specialist_site_type_code'], how='inner')

df_perMea.columns

df_perMea.isna().sum()
df_perMea.replace([np.inf, -np.inf], 0, inplace=True)

Creating the performance measure scores
### replace weightings - DMS, ESS specific 

In [29]:
df_perMea = df_perMea.fillna(0)
df_perMea = df_perMea.groupby(['Contract_ID',  'Site_Name',  'specialist_site_type_code'])['actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path', 'exPer_13_full',
       'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast', 'exPer_52_full',
       'exPer_52_path'].apply(lambda x : x.astype(int).sum())

# replace inf with 0 

df_perMea['13_full'] = round(df_perMea['actual_13_full']/df_perMea['exPer_13_full'])
df_perMea['26_full'] = round(df_perMea['actual_26_full']/df_perMea['exPer_26_full'])
df_perMea['26_path'] = round(df_perMea['actual_26_path']/df_perMea['exPer_26_path'])
df_perMea['26_wrkast'] = round(df_perMea['actual_26_wrkast']/df_perMea['exPer_26_wrkast'])
df_perMea['26_comb'] = round((df_perMea['26_full']) + (df_perMea['26_path']) + (df_perMea['26_wrkast']), ndigits=2) # combined ratio - using DMS weightings 

df_perMea['52_full'] = round(df_perMea['actual_52_full']/df_perMea['exPer_52_full'])
df_perMea['52_path'] = round(df_perMea['actual_52_path']/df_perMea['exPer_52_path'])
df_perMea['52_comb'] = round((df_perMea['52_full'] * 0.25) + (df_perMea['52_path'])) # combined ratio - using DMS weightings 

# replace infinity -> 
df_perMea.replace([np.inf, -np.inf], 0, inplace=True)
df_perMea.columns

Index(['actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path', 'exPer_13_full',
       'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast', 'exPer_52_full',
       'exPer_52_path', '13_full', '26_full', '26_path', '26_wrkast',
       '26_comb', '52_full', '52_path', '52_comb'],
      dtype='object')

In [30]:
df_perMea

actual_13_full  \
Contract_ID Site_Name                    specialist_site_type_code                   
0212789F    AimBig Employment NORWOOD    MENH                                  100   
            AimBig Employment PROSPECT   MENH                                   66   
0212790K    AimBig Employment NORWOOD    MUSK                                  100   
            AimBig Employment PROSPECT   MUSK                                   75   
0212791K    AimBig Employment MILE END   MENH                                   61   
...                                                                            ...   
0212903C    Aimbig Employment RICHMOND   AALL                                    0   
            Aimbig Employment WINDSOR    AALL                                  100   
0212904D    AimBig Employment HURSTVILLE AALL                                   68   
            AimBig Employment MIRANDA    AALL                                   66   
            AimBig Employment ROCKDALE   AALL                                   57   

                                                                    actual_26_full  \
Contract_ID Site_Name                    specialist_site_type_code                   
0212789F    AimBig Employment NORWOOD    MENH                                  100   
            AimBig Employment PROSPECT   MENH                                   61   
0212790K    AimBig Employment NORWOOD    MUSK                                  100   
            AimBig Employment PROSPECT   MUSK                                  100   
0212791K    AimBig Employment MILE END   MENH                                   66   
...                                                                            ...   
0212903C    Aimbig Employment RICHMOND   AALL                                    0   
            Aimbig Employment WINDSOR    AALL                                    0   
0212904D    AimBig Employment HURSTVILLE AALL                                   70   
            AimBig Employment MIRANDA    AALL                                   60   
            AimBig Employment ROCKDALE   AALL                                    0   

                                                                    actual_26_path  \
Contract_ID Site_Name                    specialist_site_type_code                   
0212789F    AimBig Employment NORWOOD    MENH                                    0   
            AimBig Employment PROSPECT   MENH                                    0   
0212790K    AimBig Employment NORWOOD    MUSK                                    0   
            AimBig Employment PROSPECT   MUSK                                    0   
0212791K    AimBig Employment MILE END   MENH                                   25   
...                                                                            ...   
0212903C    Aimbig Employment RICHMOND   AALL                                    0   
            Aimbig Employment WINDSOR    AALL                                    0   
0212904D    AimBig Employment HURSTVILLE AALL                                    0   
            AimBig Employment MIRANDA    AALL                                    0   
            AimBig Employment ROCKDALE   AALL                                    0   

                                                                    actual_26_wrkast  \
Contract_ID Site_Name                    specialist_site_type_code                     
0212789F    AimBig Employment NORWOOD    MENH                                      0   
            AimBig Employment PROSPECT   MENH                                      0   
0212790K    AimBig Employment NORWOOD    MUSK                                      0   
            AimBig Employment PROSPECT   MUSK                                      0   
0212791K    AimBig Employment MILE END   MENH                                      0   
...                                                                              ...   
0212903

### Step 4: Standardise the performance measure scores
Ratio of actual rate pre-standardisation and post-standardisation.

    - scale 0 to 4
    - Using MinMaxScalar

In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,4))
# subset the dataframe, then rejoin -> RESET_INDEX first to transform into dataframe, then rejoin, then groupBy




scaler.fit_transform(df_perMea.loc[:, ~df_perMea.columns.isin(['actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path', 'exPer_13_full',
       'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast', 'exPer_52_full',
       'exPer_52_path'])])

df_perMea


actual_13_full  \
Contract_ID Site_Name                    specialist_site_type_code                   
0212789F    AimBig Employment NORWOOD    MENH                                  100   
            AimBig Employment PROSPECT   MENH                                   66   
0212790K    AimBig Employment NORWOOD    MUSK                                  100   
            AimBig Employment PROSPECT   MUSK                                   75   
0212791K    AimBig Employment MILE END   MENH                                   61   
...                                                                            ...   
0212903C    Aimbig Employment RICHMOND   AALL                                    0   
            Aimbig Employment WINDSOR    AALL                                  100   
0212904D    AimBig Employment HURSTVILLE AALL                                   68   
            AimBig Employment MIRANDA    AALL                                   66   
            AimBig Employment ROCKDALE   AALL                                   57   

                                                                    actual_26_full  \
Contract_ID Site_Name                    specialist_site_type_code                   
0212789F    AimBig Employment NORWOOD    MENH                                  100   
            AimBig Employment PROSPECT   MENH                                   61   
0212790K    AimBig Employment NORWOOD    MUSK                                  100   
            AimBig Employment PROSPECT   MUSK                                  100   
0212791K    AimBig Employment MILE END   MENH                                   66   
...                                                                            ...   
0212903C    Aimbig Employment RICHMOND   AALL                                    0   
            Aimbig Employment WINDSOR    AALL                                    0   
0212904D    AimBig Employment HURSTVILLE AALL                                   70   
            AimBig Employment MIRANDA    AALL                                   60   
            AimBig Employment ROCKDALE   AALL                                    0   

                                                                    actual_26_path  \
Contract_ID Site_Name                    specialist_site_type_code                   
0212789F    AimBig Employment NORWOOD    MENH                                    0   
            AimBig Employment PROSPECT   MENH                                    0   
0212790K    AimBig Employment NORWOOD    MUSK                                    0   
            AimBig Employment PROSPECT   MUSK                                    0   
0212791K    AimBig Employment MILE END   MENH                                   25   
...                                                                            ...   
0212903C    Aimbig Employment RICHMOND   AALL                                    0   
            Aimbig Employment WINDSOR    AALL                                    0   
0212904D    AimBig Employment HURSTVILLE AALL                                    0   
            AimBig Employment MIRANDA    AALL                                    0   
            AimBig Employment ROCKDALE   AALL                                    0   

                                                                    actual_26_wrkast  \
Contract_ID Site_Name                    specialist_site_type_code                     
0212789F    AimBig Employment NORWOOD    MENH                                      0   
            AimBig Employment PROSPECT   MENH                                      0   
0212790K    AimBig Employment NORWOOD    MUSK                                      0   
            AimBig Employment PROSPECT   MUSK                                      0   
0212791K    AimBig Employment MILE END   MENH                                      0   
...                                                                              ...   
0212903

### Step 5: Calculate the Overall Performance Score
- Multiplying the standardised performance scores with the DMS/ESS weighting
TO-DO: multiply with the respective DMS/ESS weight

In [32]:
df_perMea

actual_13_full  \
Contract_ID Site_Name                    specialist_site_type_code                   
0212789F    AimBig Employment NORWOOD    MENH                                  100   
            AimBig Employment PROSPECT   MENH                                   66   
0212790K    AimBig Employment NORWOOD    MUSK                                  100   
            AimBig Employment PROSPECT   MUSK                                   75   
0212791K    AimBig Employment MILE END   MENH                                   61   
...                                                                            ...   
0212903C    Aimbig Employment RICHMOND   AALL                                    0   
            Aimbig Employment WINDSOR    AALL                                  100   
0212904D    AimBig Employment HURSTVILLE AALL                                   68   
            AimBig Employment MIRANDA    AALL                                   66   
            AimBig Employment ROCKDALE   AALL                                   57   

                                                                    actual_26_full  \
Contract_ID Site_Name                    specialist_site_type_code                   
0212789F    AimBig Employment NORWOOD    MENH                                  100   
            AimBig Employment PROSPECT   MENH                                   61   
0212790K    AimBig Employment NORWOOD    MUSK                                  100   
            AimBig Employment PROSPECT   MUSK                                  100   
0212791K    AimBig Employment MILE END   MENH                                   66   
...                                                                            ...   
0212903C    Aimbig Employment RICHMOND   AALL                                    0   
            Aimbig Employment WINDSOR    AALL                                    0   
0212904D    AimBig Employment HURSTVILLE AALL                                   70   
            AimBig Employment MIRANDA    AALL                                   60   
            AimBig Employment ROCKDALE   AALL                                    0   

                                                                    actual_26_path  \
Contract_ID Site_Name                    specialist_site_type_code                   
0212789F    AimBig Employment NORWOOD    MENH                                    0   
            AimBig Employment PROSPECT   MENH                                    0   
0212790K    AimBig Employment NORWOOD    MUSK                                    0   
            AimBig Employment PROSPECT   MUSK                                    0   
0212791K    AimBig Employment MILE END   MENH                                   25   
...                                                                            ...   
0212903C    Aimbig Employment RICHMOND   AALL                                    0   
            Aimbig Employment WINDSOR    AALL                                    0   
0212904D    AimBig Employment HURSTVILLE AALL                                    0   
            AimBig Employment MIRANDA    AALL                                    0   
            AimBig Employment ROCKDALE   AALL                                    0   

                                                                    actual_26_wrkast  \
Contract_ID Site_Name                    specialist_site_type_code                     
0212789F    AimBig Employment NORWOOD    MENH                                      0   
            AimBig Employment PROSPECT   MENH                                      0   
0212790K    AimBig Employment NORWOOD    MUSK                                      0   
            AimBig Employment PROSPECT   MUSK                                      0   
0212791K    AimBig Employment MILE END   MENH                                      0   
...                                                                              ...   
0212903

In [33]:
df_perMea['13_full'] = df_perMea['13_full'] * 0.20
df_perMea['26_full'] = df_perMea['26_full'] * 0.40
df_perMea['52_comb'] = df_perMea['52_comb'] * 0.25

df_perMea['overall_perScore'] = df_perMea['13_full'] + df_perMea['26_full'] + df_perMea['52_comb']

### Step 6: Calculate the Star Rating Percentage
- Calculated using the formula:
str% = (performance score for contract X) - (National Average contract perf score)/(National Average contract perf score) * 100 


In [34]:
# bringing in the sub368 file to read in the national averages
df_avg = pd.read_csv("C:/Users/AManalo/star_ratings_new/Sub368_Data_Sep2022.csv")
df_avg.columns

Index(['REPORT_DATE', 'EOM_SEQ', 'EOW_SEQ', 'DES_PROGRAM', 'ORGANISATION_NAME',
       'ORGANISATION_CODE', 'SUMMARY_LINE_TYPE', 'LMR_NAME', 'LMR_CODE',
       'ESA_NAME', 'ESA_CODE', 'CONTRACT_ID', 'SITE_NAME', 'SITE_CODE',
       'LOCALITY', 'SITE_TYPE', 'SPECIALTY_CD', 'START_DATE', 'END_DATE',
       'PROV_EOM_SEQ', 'PROV_EOW_SEQ', 'CONTRACT_START_GROUP',
       'PERFORMANCE_MEASURES', 'RATE', 'NUMERATOR', 'DENOMINATOR',
       'OWN_ORG_ESA_AVG', 'OWN_ORG_LMR_AVG', 'OWN_ORG_NAT_AVG',
       'ALL_ORG_SAME_SPEC_ESA_AVG', 'ALL_ORG_SAME_SPEC_LMR_AVG',
       'ALL_ORG_SAME_SPEC_NAT_AVG', 'ALL_ORG_ESA_AVG', 'ALL_ORG_LMR_AVG',
       'ALL_ORG_NAT_AVG', 'NAT_AVG_ALL_SPEC_JULY_2018'],
      dtype='object')

Joining the sub368 file with the current dataframe to get the national average

str% = (performance score for contract X) - (National Average contract perf score)/(National Average contract perf score) * 100 

In [35]:
df_perMea = df_perMea.reset_index()

In [36]:
df_avg

,REPORT_DATE,EOM_SEQ,EOW_SEQ,DES_PROGRAM,ORGANISATION_NAME,ORGANISATION_CODE,SUMMARY_LINE_TYPE,LMR_NAME,LMR_CODE,ESA_NAME,...,OWN_ORG_ESA_AVG,OWN_ORG_LMR_AVG,OWN_ORG_NAT_AVG,ALL_ORG_SAME_SPEC_ESA_AVG,ALL_ORG_SAME_SPEC_LMR_AVG,ALL_ORG_SAME_SPEC_NAT_AVG,ALL_ORG_ESA_AVG,ALL_ORG_LMR_AVG,ALL_ORG_NAT_AVG,NAT_AVG_ALL_SPEC_JULY_2018
0,30/09/2022,0.0,5.0,DES - DMS,AIMBIG EMPLOYMENT PTY LTD,VVFE,OUTLET,Adelaide,4ADE,Eastern Adelaide SA,...,0.0,100.0,64.7,0.0,0.0,42.9,70.0,68.8,73.7,42.9
1,30/09/2022,0.0,5.0,DES - DMS,AIMBIG EMPLOYMENT PTY LTD,VVFE,OUTLET,Adelaide,4ADE,Eastern Adelaide SA,...,0.0,100.0,64.7,0.0,0.0,42.9,50.0,66.3,71.9,42.9
2,30/09/2022,0.0,5.0,DES - DMS,AIMBIG EMPLOYMENT PTY LTD,VVFE,OUTLET,Adelaide,4ADE,Eastern Adelaide SA,...,19.5,18.1,19.8,15.9,15.1,20.5,18.2,17.6,19.5,20.7
3,30/09/2022,0.0,5.0,DES - DMS,AIMBIG EMPLOYMENT PTY LTD,VVFE,OUTLET,Adelaide,4ADE,Eastern Adelaide SA,...,0.0,0.6,0.8,0.0,1.5,0.7,2.9,1.9,1.2,0.7
4,30/09/2022,0.0,5.0,DES - DMS,AIMBIG EMPLOYMENT PTY LTD,VVFE,OUTLET,Adelaide,4ADE,Eastern Adelaide SA,...,19.5,18.1,19.8,15.9,15.1,20.5,18.2,17.6,19.5,20.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df_avg = df_avg.rename(columns={'CONTRACT_ID' : 'Contract_ID', 'SITE_NAME' : 'Site_Name', 'SPECIALTY_CD' : 'specialist_site_type_code'})

df_avg = df_avg[['ORGANISATION_NAME','ORGANISATION_CODE', 'ESA_NAME', 'ESA_CODE', 'Contract_ID', 'Site_Name', 'specialist_site_type_code', 'ALL_ORG_NAT_AVG']]

# join df_avg to df_perMea

df_perMea = df_perMea.merge(df_avg, on=['Contract_ID', 'Site_Name', 'specialist_site_type_code'], how='inner')



In [38]:
# calculating the star rating percentage
df_perMea['star_rating_perc'] = (df_perMea['overall_perScore'] - df_perMea['ALL_ORG_NAT_AVG'])/ (df_perMea['ALL_ORG_NAT_AVG']) * 100

In [39]:
df_perMea.to_csv('C:/Users/AManalo/star_ratings_new/star_ratings.csv')